In [1]:
import re
from itertools import chain
from tqdm import tqdm
import numpy as np
import collections

DICT = []

STATE_WORD = 0#статус строки (слово)
STATE_TYPE = 1
STATE_MEANING_START = 2

class DictProcessor():
    def __init__(self): #конструктор
        self.word = ""
        self.meaning = ""
        
        self.state = STATE_WORD
        
        self.dict = []

    def update(self, line):#определяем тип строки
        if line == "":
            self.go(STATE_WORD)
            return    
        elif re.match(r"^\s*\d+\.(.*)", line):
            m = re.match(r"^\s*\d+\.(.*)", line)
            line = m.group(1)
            self.go(STATE_TYPE)
        elif re.match(r"^\s*\d+\)", line):   
            m = re.match(r"^\s*\d+\)(.*)", line)
            line = m.group(1)
            self.go(STATE_MEANING_START)
        
        self._process(line)
        
    def _process(self, line):#сканируем строчку
        if self.state == STATE_WORD:
            self.word = line
            self.go(STATE_TYPE)
        elif self.state == STATE_TYPE:
            pass
            self.go(STATE_MEANING_START)
        elif self.state == STATE_MEANING_START:
            self.meaning += " " + line.strip()
    
    def _publish(self):#записываем в память
        self.meaning = self.meaning.strip()
        if self.word != "" and self.meaning != "" and len(self.word) >= 3:
            if "см. " in self.meaning or ":" in self.meaning or "*" in self.meaning: 
                pass
            else:
                self.dict.append((self.word, self.meaning))
            
            #print("!!!| [%s] - [%s]" % (self.word, self.meaning))        
        self.meaning = ""                        
    
    def go(self, new_state):#если меняем статус с: то публикуем
        if self.state == STATE_MEANING_START:
            self._publish()
        
        self.state = new_state   
        
    def post_process(self):#в помойку многозначные слова
        counts = collections.Counter((w_m[0] for w_m in self.dict))
        
        self.dict = list(filter(lambda w_m: counts[w_m[0]] <= 3, self.dict))        

processor = DictProcessor()        
        
with open("efremova.txt") as fin:#читаем по строчно файл
    row = -1
    
    state = STATE_WORD
    
    word = ""
    for line in fin:
        row += 1
        
        if row < 3:
            continue
            
        if row > 100 and False:
            break
        
        line = line.strip()                
        
        #print("%d: %s" % (processor.state, line))
        
        processor.update(line)                        
        
processor.post_process()        

In [2]:
print(len(processor.dict))

108520


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer#текст в фичи

In [4]:
NGRAMS = (2,4)
words_tr = TfidfVectorizer(analyzer='char', ngram_range=NGRAMS, max_features=20000, use_idf=False)#настраиваем анализатор
f_words = words_tr.fit_transform((w_m[0] for w_m in processor.dict))#матрица слова/нграммам состоит из чисел кол-во нграмм в слове/кол-во слов с этой нграммой

In [5]:
import pymorphy2 #нормализатор
morph = pymorphy2.MorphAnalyzer()
cache = {}

def preprocess(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0].normal_form
        
    return cache[word]    

def tokenizer(line):
    res = []
    for w in re.findall("\w+", line):
        res.append(preprocess(w))
        
    return res    

meaning_tr = TfidfVectorizer(tokenizer=tokenizer, max_features=20000)
f_meaning = meaning_tr.fit_transform((w_m[1] for w_m in processor.dict))#аналогично определения / слова

In [6]:
for i, (k, v) in enumerate(cache.items()):
    print("%s-%s" % (k, v))
    if i > 20:
        break

часть-часть
светильника-светильник
обычно-обычно
в-в
виде-вид
колпака-колпак
предназначенная-предназначить
для-для
сосредоточения-сосредоточение
и-и
отражения-отражение
света-свет
защиты-защита
глаз-глаз
от-от
его-он
воздействия-воздействие
устар-устарый
козырек-козырёк
надеваемый-надевать
на-на
лоб-лоб


In [7]:
for i, w in enumerate(meaning_tr.get_feature_names()):
    if i > 100:
        print(w)
    if i > 300:
        break

абрис
абсолютизм
абсолютно
абсолютный
абстрактный
абстракционизм
абстракция
абхазо
авангардизм
аванс
авантюрин
авантюрист
аварец
авария
август
авиабомба
авиадесант
авиамодель
авиамодельный
авиапочта
авиационный
авиация
австралийский
австралия
австрийский
австрия
австро
австронезийский
автобиография
автобус
автогенный
автозавод
автомат
автоматизация
автоматизированный
автоматика
автоматически
автоматический
автомашина
автомобиль
автомобильный
автономный
автор
авторитарный
авторитет
авторитетный
авторский
авторство
автотипия
автотранспортный
автотурист
агата
агент
агитатор
агитационный
агитация
агитировать
агностицизм
агрегат
агрессивно
агрессия
агромелиорация
агрономия
агротехник
агрофизика
агрохимия
агул
ада
адамант
адвокат
административно
административный
администратор
администрация
адмирал
адрес
адресат
адресовать
адъюнкт
адъютант
адыгея
адыгский
ажиотаж
азарт
азартный
азбука
азербайджан
азия
азовский
азот
азотистый
азотный
аист
аистообразный
айва
айсор
академия
акация
аквамарин
аква

In [8]:
print(f_meaning.shape)#слова - фичи (кол-во)

(108520, 20000)


In [9]:
print(f_meaning[2, :])

  (0, 7981)	0.173706018898
  (0, 16689)	0.328921268814
  (0, 7418)	0.305708847973
  (0, 15574)	0.424059348588
  (0, 7759)	0.374791378393
  (0, 5208)	0.218445370783
  (0, 18935)	0.446399492569
  (0, 5539)	0.448211247927


In [10]:
#load tf info
from collections import defaultdict#словарик со знач по умолчанию

words_tf = defaultdict(lambda: 0)
with open("tf.txt") as fin:#частотный словарик
    for line in fin:
        tf, word = line.strip().split()
        tf = int(tf)
        
        words_tf[preprocess(word)] += tf
        
f_words_tf = np.fromiter((words_tf[w_m[0]] for w_m in processor.dict), dtype=int)
print(f_words_tf.shape)      

(108520,)


In [11]:
"""
words_tf = None
cache = None

import gc
gc.collect()
"""

'\nwords_tf = None\ncache = None\n\nimport gc\ngc.collect()\n'

In [12]:
BAD_W = 1

from scipy.sparse import vstack, hstack#нужно для склейки матриц
from random import randint

X = []#строчки с парами
Y = []#правильность пары

for i in tqdm(range(f_words.shape[0] // 1)):
    f_w = f_words[i:i+1, :]#матрица равная i- ой строке (строка - слово)
    f_m = f_meaning[i:i+1, :]
        
    Y.append(1)
    X.append(hstack([f_w, f_m]))
    for j in range(BAD_W):#накидываем неправильные пары
        while True:
           rand_i = randint(0, f_words.shape[0] - 1)
           if rand_i != i:
                break
        Y.append(0)    
        X.append(hstack([f_w, f_meaning[rand_i:rand_i+1, :]]))
        
    #break    
X = vstack(X)# преобразовываем тип данных    
Y = np.array(Y)

100%|██████████| 108520/108520 [01:24<00:00, 1278.62it/s]


In [13]:
print(X.shape)
print(Y.shape)
print(f_words.shape)
print(f_meaning.shape)

(217040, 40000)
(217040,)
(108520, 20000)
(108520, 20000)


In [14]:
from sklearn.model_selection import train_test_split#"рандомно" делит Х на две части
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [15]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [16]:
t = "svm"

if t == "svm":
    svm = LinearSVC()
    clf = CalibratedClassifierCV(svm) #призываем свм с вероятностью
    
clf.fit(X_train, y_train)#тренеровка

y_proba = clf.predict_proba(X_test)#предсказание

In [17]:
from sklearn.metrics import roc_auc_score
print(y_proba)
print(y_test)
print(roc_auc_score(y_test, y_proba[:, 1]))#~количество угадываний / попыток (какие мы крутые)#насколько

[[ 0.59605007  0.40394993]
 [ 0.54209524  0.45790476]
 [ 0.56158519  0.43841481]
 ..., 
 [ 0.52554686  0.47445314]
 [ 0.49539772  0.50460228]
 [ 0.52478599  0.47521401]]
[1 0 0 ..., 1 0 0]
0.714006012253


In [18]:
y_res = clf.predict_proba(X)[:, 1]#[1::BAD_W + 1]#применим ко все парам 
y_test = Y#[1::BAD_W + 1]

In [19]:
#0.9 0.2 0.4 0.5
#1 1 0 0

freq = np.repeat(f_words_tf.astype(float) / f_words_tf.max(), 2)#нормализуем частоты
freq[np.repeat(f_words_tf, 2) == 0] = 1.0#слова у которых нет частоты

#freq = 0.0

print(freq)

diff = np.abs(y_res - y_test) - freq#тест это 0 или 1 рез вероятность
diff_order = np.argsort(diff)[::-1] #diff[diff_order] - decreasing#сортируем массив

N = 100

#print(diff[diff_order])
print(diff[diff_order[:N]])
print(y_res[diff_order[:N]])
print(y_test[diff_order[:N]])

docs_nums = diff_order[:N] // (1 + BAD_W)#переходим к индексам с правильными определениями
print(docs_nums)

for i in docs_nums:
    print("%s - %s" % processor.dict[i])

[  3.64993106e-04   3.64993106e-04   3.64993106e-04 ...,   1.00000000e+00
   1.00000000e+00   1.00000000e+00]
[ 0.81756086  0.81645201  0.81460499  0.80751504  0.80411249  0.80168428
  0.79852901  0.79807457  0.79642474  0.79212828  0.79125524  0.7883077
  0.78704243  0.78513808  0.78477654  0.78360254  0.78306068  0.78275538
  0.78243115  0.781652    0.78108974  0.78098479  0.77973105  0.77954812
  0.77908219  0.77869447  0.77841199  0.77839047  0.77808815  0.77569898
  0.77433909  0.7726345   0.77199198  0.77137873  0.771298    0.7706058
  0.76998589  0.76923829  0.76830706  0.76824582  0.76769261  0.76766065
  0.76763357  0.76761655  0.76728302  0.76690257  0.76583023  0.76525771
  0.76430746  0.76403163  0.76389333  0.76342006  0.76333765  0.76298833
  0.7622985   0.76192236  0.76184619  0.76040467  0.76025205  0.75983113
  0.75875224  0.75829478  0.75702397  0.75695793  0.75679216  0.75677765
  0.7567706   0.75667849  0.75665271  0.75628815  0.7559838   0.75555381
  0.755308    0.